### Prepare wind data for LANL's LOM by binning wind speed and wind direction values 

In [1]:
import math
import numpy as np
import pandas as pd
import time
import h5pyd
import geopandas as gpd
from dw_tap.data_fetching import getData
from dw_tap.power_output import estimate_power_output
from dw_tap.lom import run_lom

#z_turbine = 30 # turbine height in [m]
#lat, lon = 39.32856, -89.40238
#obstacle_file = "../sites/180-5BuildingsManual.geojson"

#Read in data from CSV file instead of hsds + getData
atmospheric_df = pd.read_csv("../data/180_1year_12hourgranularity.csv")

# Temporary (later need to resave csv without index column) 
if "Unnamed: 0" in atmospheric_df.columns:
    atmospheric_df.drop(columns=["Unnamed: 0"], inplace=True)

/Users/dduplyak/anaconda3/envs/dw-tap/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
atmospheric_df

,datetime,ws,wd,temp,pres
0,2007-01-01 00:00:00,7.744876,232.969856,282.541870,98493.965337
1,2007-01-01 12:00:00,9.165510,278.610485,272.009949,99037.753033
2,2007-01-02 00:00:00,5.667914,294.372393,275.522644,100177.180992
3,2007-01-02 12:00:00,1.196242,199.775672,272.208313,100652.980804
4,2007-01-03 00:00:00,6.783602,194.172807,276.606049,100410.263947
...,...,...,...,...,...
726,2007-12-30 00:00:00,3.874950,176.357785,272.583710,99716.562444
727,2007-12-30 12:00:00,4.888072,152.960520,271.582703,99367.990863
728,2007-12-31 00:00:00,6.975717,191.885982,275.272369,98836.610143
729,2007-12-31 12:00:00,4.802318,227.441046,270.859406,98993.799386


In [3]:
def wd_to_bin_avg(v, wd_bins):
    for b in wd_bins:
        lb, hb = b[0], b[1]
        if v >= lb and v < hb:
            return (lb + hb) / 2.0
    if v == wd_bins[-1][1]:
        lb, hb = wd_bins[-1][0], wd_bins[-1][1]
        return (lb + hb) / 2.0
    raise ValueError("Given wind direction value (%.3f) doesn't fit any of the created bins" % v)
    return np.nan

def ws_to_bin_avg(v, ws_bins):
    for b in ws_bins:
        lb, hb = b[0], b[1]
        if v >= lb and v < hb:
            return (lb + hb) / 2.0
    if v == ws_bins[-1][1]:
        lb, hb = ws_bins[-1][0], ws_bins[-1][1]
        return (lb + hb) / 2.0
    raise ValueError("Given wind speed value (%.3f) doesn't fit any of the created bins" % v)
    return np.nan

def wind_binning(atmospheric_df, wd_bin_width=10.0, ws_bin_width=1.0):

    wd_bin_stops = np.linspace(0, 360, int(360 / wd_bin_width) + 1, endpoint=True)
    wd_bins = [(wd_bin_stops[i], wd_bin_stops[i+1]) for i in range(len(wd_bin_stops)-1)]

    ws_max = math.ceil(atmospheric_df["ws"].max())
    ws_bin_stops = np.linspace(0, ws_max, int(ws_max / ws_bin_width) + 1, endpoint=True)
    ws_bins = [(ws_bin_stops[i], ws_bin_stops[i+1]) for i in range(len(ws_bin_stops)-1)]
    
    atmospheric_df_binned = atmospheric_df.copy()
    atmospheric_df_binned["wd"] = atmospheric_df_binned["wd"].apply(lambda x: ws_to_bin_avg(x, wd_bins))
    atmospheric_df_binned["ws"] = atmospheric_df_binned["ws"].apply(lambda x: ws_to_bin_avg(x, ws_bins))
    return atmospheric_df_binned

atmospheric_df_binned = wind_binning(atmospheric_df, wd_bin_width=10.0, ws_bin_width=1.0)
atmospheric_df_binned

,datetime,ws,wd,temp,pres
0,2007-01-01 00:00:00,7.5,235.0,282.541870,98493.965337
1,2007-01-01 12:00:00,9.5,275.0,272.009949,99037.753033
2,2007-01-02 00:00:00,5.5,295.0,275.522644,100177.180992
3,2007-01-02 12:00:00,1.5,195.0,272.208313,100652.980804
4,2007-01-03 00:00:00,6.5,195.0,276.606049,100410.263947
...,...,...,...,...,...
726,2007-12-30 00:00:00,3.5,175.0,272.583710,99716.562444
727,2007-12-30 12:00:00,4.5,155.0,271.582703,99367.990863
728,2007-12-31 00:00:00,6.5,195.0,275.272369,98836.610143
729,2007-12-31 12:00:00,4.5,225.0,270.859406,98993.799386


In [4]:
len(atmospheric_df_binned.groupby(["ws", "wd"]))

278

In [5]:
# Number of times each unique combination of bin averages is present in the entire timeseries
np.array([len(grp) for idx, grp in atmospheric_df_binned.groupby(["ws", "wd"])])

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 5, 3, 1, 2,
       1, 2, 1, 3, 1, 2, 1, 1, 2, 1, 5, 1, 2, 4, 1, 1, 1, 2, 3, 1, 3, 4,
       1, 2, 2, 3, 1, 1, 3, 2, 1, 4, 2, 2, 3, 2, 2, 1, 1, 1, 1, 2, 1, 1,
       4, 1, 2, 1, 2, 3, 1, 6, 2, 3, 2, 3, 2, 1, 4, 5, 3, 5, 5, 5, 1, 3,
       4, 3, 3, 2, 3, 3, 1, 4, 3, 4, 4, 2, 4, 1, 8, 2, 4, 1, 6, 3, 3, 5,
       2, 6, 6, 3, 5, 5, 5, 7, 4, 1, 5, 7, 3, 2, 3, 3, 2, 6, 1, 7, 1, 5,
       5, 1, 2, 3, 1, 3, 2, 2, 2, 3, 6, 2, 4, 5, 4, 9, 3, 5, 7, 5, 5, 2,
       4, 4, 1, 3, 3, 4, 5, 5, 7, 2, 3, 2, 3, 3, 1, 2, 1, 1, 1, 4, 4, 2,
       2, 6, 1, 9, 3, 6, 6, 5, 4, 3, 4, 6, 1, 4, 3, 3, 8, 6, 3, 3, 3, 1,
       1, 3, 2, 1, 2, 1, 1, 3, 2, 3, 5, 6, 4, 2, 2, 5, 3, 2, 2, 3, 7, 4,
       2, 3, 2, 2, 1, 2, 1, 2, 1, 3, 4, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 3, 5, 2, 6, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])